---
title: "Digital Waveguide Models"
author: "Brian Cruz"
date: "2025-12-06"
categories: sound synthesis, mathematics
---


::: {.content-hidden}
# HIDDEN: Frontend mattter
:::

In [1]:
# | echo: false
# | output: false

# We want to initialize Plotly's notebook mode so that it is rendered properly in Quarto.
# Otherwise, we may see the message
#   Unable to display output for mime type(s): application/vnd.plotly.v1+json

import plotly.offline as py

py.init_notebook_mode(connected=True)


::: {.content-hidden}
# HIDDEN: Article START
:::

Digital waveguide models are a powerful technique for simulating the behavior of vibrating strings and other wave-propagating systems in a computationally efficient manner.

The 1-dimensional wave equation describes how waves propagate along a string. In a digital waveguide model, we discretize this equation to create a system of delay lines and filters that simulate the wave's travel along the string. The delay lines represent the time it takes for waves to travel back and forth along the string, while the filters model the physical properties of the string, such as stiffness and damping.

By adjusting the parameters of the delay lines and filters, we can simulate different types of strings and their behaviors, allowing for realistic sound synthesis in musical applications.

The 1D wave equation is given by:

$$\frac{\partial^2 y(x,t)}{\partial t^2} = c^2 \frac{\partial^2 y(x,t)}{\partial x^2}$$
where $y(x,t)$ is the displacement of the string at position $x$ and time $t$, and $c$ is the wave speed on the string.

In this notebook we'll use a 4th order Runge-Kutta method to numerically solve the wave equation and simulate the behavior of a vibrating string.

We'll use the following sets of initial conditions:
- Plucked string:
    - Initial displacement: A plucked string shape, such as a triangular or Gaussian profile.
    - Initial velocity: Zero initial velocity, meaning the string starts from rest.
- Struck string:
    - Initial displacement: Zero initial displacement, meaning the string starts at rest.
    - Initial velocity: A localized velocity profile, such as a Gaussian or rectangular shape, representing the impact of a hammer strike.

In [87]:
# Create a dataclass for the initial conditions of the waveguide

from dataclasses import dataclass
import numpy as np


@dataclass
class WaveguideStates:
    positions: np.ndarray
    velocities: np.ndarray
    time: float = 0.0
    linear_density: float = 0.01  # kg/m
    tension: float = 100.0  # N

    def __init__(
        self,
        positions: np.ndarray,
        velocities: np.ndarray,
        time: float = 0.0,
    ):
        # If positions and velocities are 1D arrays, convert them to 2D column vectors
        if positions.ndim == 1:
            positions = positions[np.newaxis, :]
        if velocities.ndim == 1:
            velocities = velocities[np.newaxis, :]

        self.positions = positions
        self.velocities = velocities
        self.time = time
        self.__post_init__()

    def __post_init__(self):
        if self.positions.shape != self.velocities.shape:
            raise ValueError(
                "Positions and velocities must have the same shape."
                f"Got {self.positions.shape} and {self.velocities.shape}."
            )

    @classmethod
    def initialize(cls, length: int, time: float) -> "WaveguideStates":
        positions = np.zeros((1, length), dtype=np.float64)
        velocities = np.zeros((1, length), dtype=np.float64)
        return cls(positions=positions, velocities=velocities, time=time)

    @classmethod
    def initial_pluck(
        cls, length: int, pluck_position: int, pluck_amplitude: float
    ) -> "WaveguideStates":
        positions = np.zeros(length)
        velocities = np.zeros(length)

        # Create a triangular pluck shape
        for i in range(length):
            if i < pluck_position:
                positions[i] = (pluck_amplitude / pluck_position) * i
            else:
                positions[i] = pluck_amplitude - (
                    pluck_amplitude / (length - pluck_position)
                ) * (i - pluck_position)
        return cls(positions=positions, velocities=velocities)

    @classmethod
    def initial_strike(
        cls,
        length: int,
        strike_position: int,
        strike_amplitude: float,
        strike_width: int,
    ) -> "WaveguideStates":
        positions = np.zeros(length)
        velocities = np.zeros(length)

        # Create a rectangular strike shape
        start = max(0, strike_position - strike_width // 2)
        end = min(length, strike_position + strike_width // 2)
        velocities[start:end] = strike_amplitude

        return cls(positions=positions, velocities=velocities)

    @classmethod
    def initial_standing_wave(
        cls, length: int, harmonic_amplitudes: dict[int, float] | None = None
    ) -> "WaveguideStates":
        positions = np.zeros(length)
        velocities = np.zeros(length)

        x = np.linspace(0, np.pi, length)

        if harmonic_amplitudes is not None:
            for harmonic, amplitude in harmonic_amplitudes.items():
                positions += amplitude * np.sin(harmonic * x)

        return cls(positions=positions, velocities=velocities)

    def plot(
        self,
        index: int = -1,
        height: int = 400,
        y_range: tuple[float, float] = (-0.06, 0.06),
    ) -> None:
        import plotly.graph_objects as go

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(y=self.positions[index], mode="lines", name="Positions")
        )
        fig.add_trace(
            go.Scatter(y=self.velocities[index], mode="lines", name="Velocities")
        )
        fig.update_layout(
            title="Waveguide State",
            xaxis_title="Sample Index",
            yaxis_title="Amplitude",
            legend_title="Legend",
            height=height,
            yaxis=dict(range=y_range),
        )
        fig.show()

    def plot_animation(
        self,
        height: int = 400,
        y_range: tuple[float, float] = (-0.08, 0.08),
        interval: int = 100,
        skip_frames: int = 10,
        num_frames: int | None = 100,
    ) -> None:
        import plotly.graph_objects as go

        positions = self.positions[::skip_frames]
        velocities = self.velocities[::skip_frames]

        if num_frames:
            positions = positions[:num_frames]
            velocities = velocities[:num_frames]

        fig = go.Figure(
            data=[
                go.Scatter(y=positions[0], mode="lines", name="Positions"),
                go.Scatter(y=velocities[0], mode="lines", name="Velocities"),
            ],
            layout=go.Layout(
                title="Waveguide State Animation",
                xaxis_title="Sample Index",
                yaxis_title="Amplitude",
                legend_title="Legend",
                height=height,
                yaxis=dict(range=y_range),
                updatemenus=[
                    dict(
                        type="buttons",
                        buttons=[
                            dict(
                                label="Play",
                                method="animate",
                                args=[
                                    None,
                                    {
                                        "frame": {"duration": interval, "redraw": True},
                                        "fromcurrent": True,
                                        "transition": {
                                            "duration": 0,
                                            #    "easing": "cubic"
                                        },
                                    },
                                ],
                            ),
                            dict(
                                label="Pause",
                                method="animate",
                                args=[
                                    [None],
                                    {
                                        "frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0},
                                    },
                                ],
                            ),
                        ],
                    )
                ],
            ),
            frames=[
                go.Frame(
                    data=[
                        go.Scatter(y=positions[k], mode="lines"),
                        go.Scatter(y=velocities[k], mode="lines"),
                    ],
                    name=f"frame{k}",
                )
                for k in range(positions.shape[0])
            ],
        )
        # Add a slider
        # sliders = [
        #     {
        #         "steps": [
        #             {
        #                 "args": [
        #                     [str(i)],
        #                     {
        #                         "frame": {"duration": interval, "redraw": True},
        #                         "mode": "immediate",
        #                     },
        #                 ],
        #                 "label": f"{state.time:.2f}s",
        #                 "method": "animate",
        #             }
        #             for i, state in enumerate(self.states)
        #         ],
        #         "transition": {"duration": 0},
        #         "x": 0,
        #         "y": -0.1,
        #         "currentvalue": {
        #             "font": {"size": 16},
        #             "prefix": "Time: ",
        #             "visible": True,
        #             "xanchor": "center",
        #         },
        #         "len": 1.0,
        #     }
        # ]
        # fig.update_layout(sliders=sliders)
        fig.update_layout(
            sliders=[
                dict(
                    steps=[
                        dict(
                            method="animate",
                            args=[
                                [f"frame{k}"],
                                {
                                    "frame": {"duration": interval, "redraw": True},
                                    "mode": "immediate",
                                },
                            ],
                            label=f"{k}",
                        )
                        for k in range(positions.shape[0])
                    ],
                    transition={"duration": 0},
                    x=0,
                    y=0,
                    currentvalue=dict(
                        font=dict(size=12), prefix="Frame: ", visible=True
                    ),
                    len=1.0,
                )
            ]
        )

        fig.show()


# Example usage
standing_wave_state = WaveguideStates.initial_standing_wave(
    length=100,
    harmonic_amplitudes={
        1: 0.03,
        2: 0.02,
        3: 0.03,
    },
)
# standing_wave_state.positions.shape
# standing_wave_state.plot()


def simulate_waveguide(
    initial_state: WaveguideStates,
    num_steps: int,
    c: float = 1.0,
    dx: float = 1.0,
    dt: float = 0.1,
) -> WaveguideStates:
    length = len(initial_state.positions[-1])
    position = initial_state.positions[-1].copy()
    velocity = initial_state.velocities[-1].copy()
    time = initial_state.time

    positions_to_append = []
    velocities_to_append = []

    for step in range(num_steps):
        new_position = position.copy()
        new_velocity = velocity.copy()

        for i in range(1, length - 1):
            new_velocity[i] += (c**2 * dt / dx**2) * (
                position[i + 1] - 2 * position[i] + position[i - 1]
            )
        # Update position using velocity and acceleration (Verlet integration)
        new_position += new_velocity * dt
        new_position += (
            0.5 * (new_velocity - velocity) * dt**2
        )  # Verlet term. Seems to add a LP filter

        position = new_position
        velocity = new_velocity
        time += dt

        # if step % 100 == 0:
        # print(f"Step {step}/{num_steps}")
        positions_to_append.append(position.copy())
        velocities_to_append.append(velocity.copy())

    new_wave_guide_states = WaveguideStates(
        positions=np.vstack([initial_state.positions] + positions_to_append),
        velocities=np.vstack([initial_state.velocities] + velocities_to_append),
        time=time,
    )

    return new_wave_guide_states


output = simulate_waveguide(standing_wave_state, num_steps=4000)


# output.positions.shape
# output.plot(index=0)
# output.plot(index=-1)
output.plot_animation()
# plucked_state = WaveguideStates.initial_pluck(
#     length=100, pluck_position=30, pluck_amplitude=0.05
# )
# plucked_state.plot()

# struck_state = WaveguideStates.initial_strike(
#     length=100, strike_position=50, strike_amplitude=0.05, strike_width=10
# )
# struck_state.plot()


In [68]:
df

,time_step,sample_index,amplitude
0,0,0,"[0.0, 0.0038035193040675094, 0.007580254920509..."
1,1,1,"[0.0, 0.003803238075347439, 0.0075796949289171..."
2,2,2,"[0.0, 0.0025235834121880214, 0.005031580103943..."
3,3,3,"[0.0, -0.0002056468610163978, -0.0004032839527..."
4,4,4,"[0.0, -0.0021965361918980897, -0.0043687663979..."
5,5,5,"[0.0, -0.0019380766843792186, -0.0038563622368..."
6,6,6,"[0.0, 0.00015212645761605585, 0.00030271713027..."
7,7,7,"[0.0, 0.0020990905484653687, 0.004176631497105..."
8,8,8,"[0.0, 0.0020503944115550132, 0.004077552138980..."
9,9,9,"[0.0, -0.0001476349221440968, -0.0003002627258..."


In [69]:
# Use plotly express for animations
import plotly.express as px
import pandas as pd


df = pd.DataFrame(
    [
        {"time_step": t, "sample_index": t, "amplitude": output.positions[t]}
        for t in range(output.positions.shape[0])
        # for i in range(output.positions.shape[1])
    ]
)
fig = px.line(
    df,
    x="sample_index",
    y="amplitude",
    animation_frame="sample_index",
    title="Waveguide Positions Over Time",
)
fig.show()
# fig = px.line(
#     output.positions,
#     # labels={"index": "Sample Index", "value": "Amplitude", "variable": "Time Step"},
#     title="Waveguide Positions Over Time",
# )
# fig.update_layout(height=400, yaxis=dict(range=(-0.06, 0.06)))
# fig.show()

In [33]:
struck_state.plot_animation(interval=50)

AttributeError: 'WaveguideStates' object has no attribute 'states'

In [ ]:
@dataclass
class WaveguideStates:
    states: list[WaveguideState]

    def append(self, state: WaveguideState):
        self.states.append(state)

    def get_state_at(self, index: int) -> WaveguideState:
        return self.states[index]

    def plot_animation(self, interval: int = 200, skip_frames: int = 1000):
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots

        fig = make_subplots()
        fig.add_trace(
            go.Scatter(y=self.states[0].position, mode="lines", name="Position")
        )
        fig.add_trace(
            go.Scatter(y=self.states[0].velocity, mode="lines", name="Velocity")
        )
        frames = [
            go.Frame(
                data=[go.Scatter(y=state.position), go.Scatter(y=state.velocity)],
                name=str(i),
            )
            for i, state in enumerate(self.states[::skip_frames])
        ]
        fig.frames = frames
        fig.update_layout(
            title="Waveguide State Animation",
            xaxis_title="Sample Index",
            yaxis_title="Amplitude",
            updatemenus=[
                {
                    "type": "buttons",
                    "buttons": [
                        {
                            "label": "Play",
                            "method": "animate",
                            "args": [
                                None,
                                {
                                    "frame": {"duration": interval, "redraw": True},
                                    "fromcurrent": True,
                                },
                            ],
                        },
                        {
                            "label": "Pause",
                            "method": "animate",
                            "args": [
                                [None],
                                {
                                    "frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "fromcurrent": True,
                                },
                            ],
                        },
                    ],
                }
            ],
            # Show time updating in the top center
            # times=[state.time for state in self.states],
            annotations=[
                {
                    "text": "Time: 0.0 s",
                    "x": 0.5,
                    "y": 1.1,
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {"size": 16},
                    "align": "center",
                    "name": "time_annotation",
                }
            ],
            # Set ylim to -0.05 to 0.05
            yaxis=dict(range=[-0.05, 0.05]),
        )

        # Add slider
        sliders = [
            {
                "steps": [
                    {
                        "args": [
                            [str(i)],
                            {
                                "frame": {"duration": interval, "redraw": True},
                                "mode": "immediate",
                            },
                        ],
                        "label": f"{state.time:.2f}s",
                        "method": "animate",
                    }
                    for i, state in enumerate(self.states)
                ],
                "transition": {"duration": 0},
                "x": 0,
                "y": -0.1,
                "currentvalue": {
                    "font": {"size": 16},
                    "prefix": "Time: ",
                    "visible": True,
                    "xanchor": "center",
                },
                "len": 1.0,
            }
        ]
        fig.update_layout(sliders=sliders)
        fig.show()

In [ ]:
# %matplotlib ipympl

In [ ]:
def simulate_waveguide(
    initial_state: WaveguideState,
    num_steps: int,
    c: float = 1.0,
    dx: float = 1.0,
    dt: float = 0.1,
) -> WaveguideStates:
    length = len(initial_state.position)
    position = initial_state.position.copy()
    velocity = initial_state.velocity.copy()
    time = initial_state.time

    states = WaveguideStates(states=[initial_state])

    for step in range(num_steps):
        new_position = position.copy()
        new_velocity = velocity.copy()

        for i in range(1, length - 1):
            new_velocity[i] += (c**2 * dt / dx**2) * (
                position[i + 1] - 2 * position[i] + position[i - 1]
            )
        # Update position using velocity and acceleration (Verlet integration)
        new_position += new_velocity * dt
        new_position += (
            0.5 * (new_velocity - velocity) * dt**2
        )  # Verlet term. Seems to add a LP filter

        position = new_position
        velocity = new_velocity
        time += dt

        if step % 1 == 0:
            states.append(
                WaveguideState(
                    position=position.copy(), velocity=velocity.copy(), time=time
                )
            )

    return states


output = simulate_waveguide(standing_wave_state, num_steps=40000)


In [ ]:
output.plot_animation(interval=50)

In [ ]:
output

In [ ]:
3

In [ ]:
import IPython.display as ipd

string_position = 43

audio = np.array(
    [state.position[string_position] for state in output.states], dtype=np.float32
)

# resample at 1/4th the rate by running a LP first

from scipy.signal import butter, lfilter, resample_poly
from scipy.signal import butter, lfilter, resample_poly

# def butter_lowpass(cutoff, fs, order=5):
#     nyq = 0.5 * fs
#     normal_cutoff = cutoff / nyq
#     b, a = butter(order, normal_cutoff, btype='low', analog=False)
#     return b, a

# def lowpass_filter(data, cutoff, fs, order=5):
#     b, a = butter_lowpass(cutoff, fs, order=order)
#     y = lfilter(b, a, data)
#     return y

# filtered_audio = lowpass_filter(audio, cutoff=20000, fs=96000, order=6)
audio = resample_poly(audio, up=1, down=4)

ipd.display(ipd.Audio(audio, rate=96000), autoplay=True)

In [ ]:
# Get a position on the string and play the sound

import IPython.display as ipd

string_position = 71

audio = np.array(
    [state.position[string_position] for state in output.states], dtype=np.float32
)
ipd.display(ipd.Audio(audio, rate=44100), autoplay=True)
# audio

In [ ]:
# Simulate a plucked string using the 1D wave equation and finite difference methods

from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio

# Parameters
fs = 44100  # Sampling frequency
duration = 2.0  # Duration in seconds
L = 1.0  # Length of the string in meters
T = 1000.0  # Tension in Newtons
mu = 0.01  # Linear mass density in kg/m
c = np.sqrt(T / mu)  # Wave speed
Nx = 100  # Number of spatial points
dx = L / (Nx - 1)  # Spatial step
dt = 1 / fs  # Time step
Nt = int(duration / dt)  # Number of time steps

# Stability condition
if c * dt / dx > 1:
    raise ValueError("Stability condition violated: c*dt/dx must be <= 1")
# Initial conditions: plucked string
x = np.linspace(0, L, Nx)
u0 = np.zeros(Nx)
pluck_position = 0.25 * L
for i in range(Nx):
    if x[i] < pluck_position:
        u0[i] = (x[i] / pluck_position) * 0.5
    else:
        u0[i] = ((L - x[i]) / (L - pluck_position)) * 0.5
u1 = np.copy(u0)
u2 = np.zeros(Nx)


# Use Runge-Kutta 4th order method for time integration
def rk4_step(u, u_prev, c, dt, dx):
    def dudt(u, u_prev):
        d2u_dx2 = np.zeros_like(u)
        d2u_dx2[1:-1] = (u[2:] - 2 * u[1:-1] + u[:-2]) / dx**2
        return c**2 * d2u_dx2

    k1 = dudt(u, u_prev)
    k2 = dudt(u + 0.5 * dt * k1, u_prev)
    k3 = dudt(u + 0.5 * dt * k2, u_prev)
    k4 = dudt(u + dt * k3, u_prev)

    return u + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)


# Time-stepping loop
output = []
for n in range(Nt):
    u_new = rk4_step(u1, u0, c, dt, dx)
    output.append(u_new[int(Nx / 2)])  # Record the midpoint displacement
    u0, u1 = u1, u_new

# Convert output to numpy array
output = np.array(output)

In [ ]:
output.shape

In [ ]:
# Solve the

In [ ]:
# # Use Pybind11 to create Python bindings for C++ code.

# from pybind11 import get_cmake_dir
# import pybind11
# import os
# from setuptools import setup, Extension
# from setuptools.command.build_ext import build_ext
# import sys
# import setuptools
# import glob

# # --- IGNORE --- `
# # from Cython.Build import cythonize
# # ` --- END IGNORE ---
# from distutils.sysconfig import get_config_var
# from distutils.version import LooseVersion
# import subprocess
# import re

::: {.content-hidden}
# HIDDEN: Article END
:::

::: {.content-hidden}
# This entire markdown section will be hidden.
:::

In [ ]:
# | echo: false
# | output: false
print("This code and its output will be hidden.")